In [1]:
import os
import torch
import numpy as np

from torch.utils.data import TensorDataset, DataLoader, Subset
from os import path

from src.models import *
from src.methods import *
from src.utils import *

data_path = path.join(os.getcwd(), 'project_data/')

USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cpu


/home/qimingliu0831/anaconda3/envs/nlp/lib/python3.10/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
X_test = np.load(data_path + "X_test.npy")
y_test = np.load(data_path + "y_test.npy") - 769
person_train_valid = np.load(data_path + "person_train_valid.npy")
X_train_valid = np.load(data_path + "X_train_valid.npy")
y_train_valid = np.load(data_path + "y_train_valid.npy") - 769
person_test = np.load(data_path + "person_test.npy")

In [3]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [4]:
X_train_valid = X_train_valid[:, np.newaxis, :, :]
X_test = X_test[:, np.newaxis, :, :]

In [5]:
X_train_valid = torch.from_numpy(X_train_valid).float()
y_train_valid = torch.from_numpy(y_train_valid).long()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()

train_valid_dataset = TensorDataset(X_train_valid, y_train_valid)
test_dataset = TensorDataset(X_test, y_test)

num_train = int(0.8 * len(train_valid_dataset))
num_valid = len(train_valid_dataset) - num_train
train_indices, valid_indices = torch.utils.data.random_split(np.arange(len(train_valid_dataset)), [num_train, num_valid])

train_dataset = Subset(train_valid_dataset, train_indices)
valid_dataset = Subset(train_valid_dataset, valid_indices)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print('Training/Valid data shape: {}'.format(X_train_valid.shape))
print('Test data shape: {}'.format(X_test.shape))

Training/Valid data shape: torch.Size([2115, 1, 22, 1000])
Test data shape: torch.Size([443, 1, 22, 1000])


In [7]:
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-7, weight_decay=0.0005)

train(model, train_loader, valid_loader, criterion, optimizer, device=device, epochs=50)

test_accuracy = evaluate(model, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

 20%|██        | 10/50 [05:01<20:15, 30.40s/it]

Epoch [10/50], Loss: 1.4135, Validation Accuracy: 33.10%


 40%|████      | 20/50 [09:58<15:10, 30.33s/it]

Epoch [20/50], Loss: 1.0982, Validation Accuracy: 43.97%


 60%|██████    | 30/50 [15:00<10:07, 30.36s/it]

...

 80%|████████  | 40/50 [19:57<05:02, 30.24s/it]

Epoch [40/50], Loss: 0.4099, Validation Accuracy: 44.92%


 82%|████████▏ | 41/50 [20:56<04:35, 30.66s/it]

Early stopping!


Test Accuracy: 42.21%


In [8]:
lstm = LSTMModel(input_size=1000)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-7, weight_decay=0.0005)

lstm = lstm.to(device)
train(lstm, train_loader, valid_loader, criterion, optimizer, device=device)

test_accuracy = evaluate(lstm, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

  0%|          | 0/20 [00:00<?, ?it/s]

 50%|█████     | 10/20 [00:12<00:11,  1.14s/it]

Epoch [10/20], Loss: 0.1329, Validation Accuracy: 31.21%


100%|██████████| 20/20 [00:25<00:00,  1.27s/it]

Epoch [20/20], Loss: 0.0722, Validation Accuracy: 29.08%
Test Accuracy: 28.67%


In [7]:
cnn_lstm = CNN_LSTM()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn_lstm.parameters(), lr = 0.001, betas=(0.9, 0.99), eps=1e-7, weight_decay=0.0005)

cnn_lstm = cnn_lstm.to(device)
train(cnn_lstm, train_loader, valid_loader, criterion, optimizer, device=device, epochs=100)

test_accuracy = evaluate(cnn_lstm, test_loader, device=device)
print(f'Test Accuracy: {test_accuracy:.2f}%')

 10%|█         | 10/100 [00:12<01:48,  1.21s/it]

Epoch [10/100], Loss: 1.1323, Validation Accuracy: 45.86%


 20%|██        | 20/100 [00:24<01:36,  1.21s/it]

Epoch [20/100], Loss: 0.4571, Validation Accuracy: 52.96%


 30%|███       | 30/100 [00:36<01:24,  1.20s/it]

Epoch [30/100], Loss: 0.2133, Validation Accuracy: 54.61%


 35%|███▌      | 35/100 [00:43<01:20,  1.24s/it]

Early stopping!
Test Accuracy: 54.85%
